https://www.kaggle.com/code/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm/comments

# 목적:
to use **historical loan application data** to predict whether or not an applicant will be able to repay a loan.

For each SK_ID_CURR in the test set, you must predict a **probability for the TARGET variable. **

1. application_train/application_test (주 대출 신청 데이터): 이 데이터 세트에는 Home Credit에서 대출 신청 시 제공된 각 대출에 대한 정보가 포함되어 있습니다. 각 대출은 SK_ID_CURR이라는 고유 번호로 식별됩니다. 대출 신청 데이터에는 대출이 상환되었는지(TARGET 값 0), 상환되지 않았는지(TARGET 값 1)를 나타내는 TARGET 필드가 포함되어 있습니다.

2. bureau (타 금융 기관 대출 정보): 이 데이터는 고객이 다른 금융 기관에서 받은 이전 대출에 관한 정보를 담고 있습니다. 이전 대출 각각이 자체 행을 가지고 있으며, 한 대출 신청 데이터는 여러 이전 대출 정보를 가질 수 있습니다.

3. bureau_balance (타 금융 기관 대출 잔액 월별 데이터): bureau에 기록된 이전 대출의 월별 데이터입니다. 각 행은 이전 대출의 한 달치를 나타내며, 하나의 이전 대출에 대해 여러 행이 있을 수 있습니다.

4. previous_application (이전 대출 신청 데이터): Home Credit에서 고객이 이전에 신청한 대출에 대한 정보입니다. 현재 대출 신청 데이터에 포함된 고객이 이전에 신청한 대출 각각에 대해 한 행씩 있으며, SK_ID_PREV로 식별됩니다.

5. POS_CASH_BALANCE (판매점 현금 대출 월별 데이터): Home Credit과의 이전 판매점 현금 대출에 대한 월별 데이터입니다. 각 행은 이전 대출의 한 달치를 나타내며, 하나의 대출에 대해 여러 행이 있을 수 있습니다.

6. credit_card_balance (신용카드 잔액 월별 데이터): 고객이 Home Credit과 가졌던 이전 신용카드의 월별 잔액 데이터입니다. 각 행은 한 달치 신용카드 잔액을 나타내며, 하나의 신용카드 계좌에 대해 여러 행이 있을 수 있습니다.

7. installments_payment (대출 상환 내역): Home Credit에서의 이전 대출에 대한 상환 내역입니다. 상환된 각각의 지불과 누락된 각각의 지불에 대해 한 행씩 있습니다.

즉, Home Credit에서 대출을 신청한 고객들의 신청 정보, 이전 대출 상환 내역, 그리고 다른 금융 기관에서의 대출 내역을 통해 Home Credit이 고객의 대출 상환 능력을 평가하고 예측하기







* 진짜 양성 비율(TPR): 실제로 양성인 데이터 중에서 모델이 양성으로 올바르게 예측한 데이터의 비율입니다. 예를 들어, 실제 스팸 이메일 100개 중 모델이 90개를 스팸으로 올바르게 예측했다면 TPR은 90%입니다.
* 거짓 양성 비율(FPR): 실제로는 음성인 데이터 중에서 모델이 양성으로 잘못 예측한
데이터의 비율입니다. 예를 들어, 실제 스팸이 아닌 이메일 100개 중 모델이 10개를 스팸으로 잘못 예측했다면 FPR은 10%입니다.

ROC AUC는 ROC 곡선 아래의 면적으로, 0에서 1 사이의 값을 가집니다. 이 값이 1에 가까울수록 모델의 성능이 뛰어나다는 것을 의미합니다. 즉, 모델이 진짜 양성을 양성으로, 진짜 음성을 음성으로 정확하게 분류할 확률이 높다는 것을 나타냅니다. 반면, AUC 값이 0.5에 가까우면 모델의 성능이 무작위 추측 수준임을 의미합니다.

In [ ]:
from google.colab import drive

In [ ]:
!unzip '/content/drive/MyDrive/KUBIG/데이터분석 스터디/home-credit-default-risk.zip'

Archive:  /content/drive/MyDrive/KUBIG/데이터분석 스터디/home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [ ]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

NFOLDS = 3
SEED = 0
NROWS = None

data = pd.read_csv('/content/application_train.csv')
test = pd.read_csv('/content/application_test.csv')
prev = pd.read_csv('/content/previous_application.csv')

categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]

for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

gc.enable()

y_train = data['TARGET']
del data['TARGET']

prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train = x_train.fillna(0)
x_test= x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

kf = KFold(n_splits = NFOLDS, shuffle=True, random_state=SEED)

class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]


class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))


def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

lightgbm_params = {
    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':123,
    'colsample_bytree':0.8,
    'subsample':0.9,
    'max_depth':15,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'min_split_gain':0.01,
    'min_child_weight':2
}

xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf= CatBoostClassifier, seed = SEED, params=catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

print("XG-CV: {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("ET-CV: {}".format(sqrt(mean_squared_error(y_train, et_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))

x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)

test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]

test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:02:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:03:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:03:37] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


0:	total: 218ms	remaining: 43.3s
1:	total: 337ms	remaining: 33.4s
2:	total: 437ms	remaining: 28.7s
3:	total: 565ms	remaining: 27.7s
4:	total: 636ms	remaining: 24.8s
5:	total: 699ms	remaining: 22.6s
6:	total: 756ms	remaining: 20.8s
7:	total: 809ms	remaining: 19.4s
8:	total: 877ms	remaining: 18.6s
9:	total: 941ms	remaining: 17.9s
10:	total: 996ms	remaining: 17.1s
11:	total: 1.07s	remaining: 16.8s
12:	total: 1.14s	remaining: 16.4s
13:	total: 1.2s	remaining: 15.9s
14:	total: 1.26s	remaining: 15.6s
15:	total: 1.32s	remaining: 15.2s
16:	total: 1.37s	remaining: 14.8s
17:	total: 1.43s	remaining: 14.4s
18:	total: 1.48s	remaining: 14.1s
19:	total: 1.54s	remaining: 13.8s
20:	total: 1.6s	remaining: 13.6s
21:	total: 1.65s	remaining: 13.3s
22:	total: 1.71s	remaining: 13.2s
23:	total: 1.76s	remaining: 12.9s
24:	total: 1.82s	remaining: 12.8s
25:	total: 1.88s	remaining: 12.6s
26:	total: 1.94s	remaining: 12.4s
27:	total: 2s	remaining: 12.3s
28:	total: 2.07s	remaining: 12.2s
29:	total: 2.13s	remaining: 1

It is just a simple starter. It can really be improved.
**First idea : increase the number of iterations in XGBOOST and number of trees in RandomForest & ExtraTrees**

The aim of this kernel was not to take care of data cleaning but focus on ensembling methods.


I am having massive overfitting - are you having the same problem? I feel like it is because of class imbalance. What do you think?
I think the overfitting part is mainly due to the second layer of the ensembling. Removing the xgb part to put some logistic regression could be way better. I plan to do it soon.


Looks like the author didn't end up using skew looking at the code. However, there is a large imbalance in the y_train values - about 10:1.
Yes exactly, I didn't use skew here but you could imagine using it to transform to log some features

In [ ]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')